# Use Pandas in ArcGIS

```{image} ../_static/images/pandaslogo.svg
:class: border
:alt: pandaslogo
:width: 400
:align: right
```

[Pandas](https://pandas.pydata.org/) is widely used in **data science** and data analysis,
and is a popular choice for _cleaning_, _transforming_, and _analyzing_ data in Python.

The name "pandas" is derived from "**panel data**", which is an econometric term for data sets
that include observations over multiple time periods for the same individuals.

- pandas is built on top of NumPy
- the "_excel spreadsheet_" in Python
- Powerful indexing selection
- easily integrated with other libraries, such as _matplotlib_, _scikit-learn_, and _plotly_.

Like NumPy, the convention of importing `pandas` is following. And as yet another
routine, the two packages are usually imported together with `numpy` being the first.

In [2]:
import numpy as np
import pandas as pd

## 1. `pandas.Series`

[Series](https://pandas.pydata.org/docs/reference/api/pandas.Series.html) is the fundamental
data structure of pandas.
A `pandas.Series` object is essentially a 1-d array of indexed values.

Let's again start by creating a `pd.Series` using a list.

In [3]:
my_list = list(range(10))
my_list

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
pd.Series(my_list) # note that the S in Series is upper case

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
dtype: int64

The above code converts a list to a Series. Note that the default index for
Series starts with index `0`.

But `pd.Series` can have non-numeric and non-sequential indices, whereas a
`np.ndarray` is only indexed by whole numbers.

Let's customize the index of this Series.

In [5]:
import string
string.ascii_lowercase[:10]

'abcdefghij'

In [6]:
labels = list(string.ascii_lowercase[:10])
labels

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

In [7]:
pd.Series(my_list, index=labels)

a    0
b    1
c    2
d    3
e    4
f    5
g    6
h    7
i    8
j    9
dtype: int64

In [9]:
my_sr = pd.Series(my_list, labels)

We can use `values` to obtain the values as a 1-d `np.ndarray`.

In [ ]:
my_sr.values

Likewise, we can also retrieve a Series's index.

In [ ]:
my_sr.index

### 1.1 Indexing and slicing for Series

- indexing by label
- indexing by location

In [10]:
my_sr['c']   # indexing by label

2

In [11]:
my_sr[2]   # indexing by position

2

In [12]:
my_sr[2:8]

c    2
d    3
e    4
f    5
g    6
h    7
dtype: int64

### 1.2 Operations on Series

Similar to `np.ndarray`, operations on `pd.Series` applies to every single element in it.

Let's create `pd.Series` to represent the following tables.

<font color='steelblue'>**Top 3 U.S. State by Population**</font>

| State | Population |
|-------|------------|
| CA    | 39512223   |
| TA    | 28995881   |
| FL    | 21477737   |

<font color='steelblue'>**Top 3 U.S. State by Area**</font>

| State | Area(sq mi) |
|-------|-------------|
| AL    | 665384      |
| TA    | 268596      |
| CA    | 163694      |

Let's use `split()`, a **string manipulation** function, to split the string
by whitespace, which will result in a `list`.

In [14]:
"CA TA FL".split(" ")

['CA', 'TA', 'FL']

In [15]:
"CA TA FL".split()  # split() acts on space by default

['CA', 'TA', 'FL']

In [16]:
sr1 = pd.Series(data=[39512223, 28995881, 21477737],
                index="CA TA FL".split())
sr1

CA    39512223
TA    28995881
FL    21477737
dtype: int64

What about population in millions?

In [17]:
round(sr1 / 1000000, 2)

CA    39.51
TA    29.00
FL    21.48
dtype: float64

Similarly, we can split a string by "comma" (or any other characters).

In [18]:
"AL,TA,CA".split(",")

['AL', 'TA', 'CA']

In [19]:
sr2 = pd.Series([665384, 268596, 163694], "AL,TA,CA".split(","))
sr2

AL    665384
TA    268596
CA    163694
dtype: int64

```{note}:
Pandas perform operations based on indicies. In other word, only values of the same
index can be used together to perform some operation.
```

In [20]:
sr1 / sr2

AL           NaN
CA    241.378566
FL           NaN
TA    107.953510
dtype: float64

Note that only California and Texas have values returned because their indices exist
in both Series. On the contrary, Alaska doesn't appear in `sr1` and Florida isn't in `sr2`,
the division for those returned `NaN` meaning "not a number."

## 2. DataFrame object

[DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html): **Two-dimensional** (rows & columns), size-mutable, potentially heterogeneous tabular data.

`DataFrame` consists of one or more `Series` that shares the same labels (indicies).

It is the **DataFrame** structure makes pandas a powerful tool for scientific data analysis.

### 2.1 Create DataFrame from Series

We can convert a `pd.Series` to a `pd.DataFrame` of a single column.
And note that the column names must be supplied as a list.

In [21]:
pd.DataFrame(sr1, columns=['population'])

,population
CA,39512223
TA,28995881
FL,21477737


`concat` function: series are supplied as **a list**

In [22]:
pd.concat([sr1, sr2], axis=1) # axis=1 means on columns

,0,1
CA,39512223.0,163694.0
TA,28995881.0,268596.0
FL,21477737.0,NaN
AL,NaN,665384.0


In [23]:
pd.concat([sr1, sr2]) # by default, axis=0, meaning on rows

CA    39512223
TA    28995881
FL    21477737
AL      665384
TA      268596
CA      163694
dtype: int64

`concat` function: series are supplied as **a dictionary**

In [24]:
pd.concat({'population': sr1, 'area (sq mi)': sr2}, axis=1)

,population,area (sq mi)
CA,39512223.0,163694.0
TA,28995881.0,268596.0
FL,21477737.0,NaN
AL,NaN,665384.0


### 2.2 Convert NumPy's Structured Array to `pd.DataFrame`

Recall how we convert a feature class to a **structured ndarray**

In [40]:
import arcpy

gdb_worksp = r"..\data\class_data.gdb"
arcpy.env.workspace = gdb_worksp
school_fc = "schools"

school_arr = arcpy.da.FeatureClassToNumPyArray(school_fc, '*') # use '*' to get all fields

In [41]:
school_arr.dtype.names

('OBJECTID_1',
 'Shape',
 'OBJECTID',
 'STATUS',
 'SCORE',
 'SIDE',
 'MATCH_ADDR',
 'FEDERAL_ID',
 'STATE_ID',
 'SCHOOL_ID',
 'NAME',
 'ADDRESS',
 'CITY',
 'ZIPCODE',
 'PHONE',
 'COUNTY',
 'OPERATING',
 'OP_CLASS',
 'ENROLLMENT',
 'PROGRAMS',
 'COMMON_USE',
 'USE',
 'TYPE',
 'ACTIVITY',
 'GRADES',
 'LOW_GRADE',
 'HIGH_GRADE',
 'PRINCIPAL',
 'TEACHERS',
 'STDTCH_RT',
 'MIGRNT_STD',
 'TITLE1SCHO',
 'MAGNETINFO',
 'FREE_LUNCH',
 'REDUCED_LU',
 'FISH_FAC1',
 'FISH_FAC2',
 'COMMENTS',
 'BBSERVICE',
 'BBPROVIDER',
 'BBSPEED',
 'DSTREAMSPD',
 'YR_BUILT',
 'PARCEL_ID',
 'LAT_DD',
 'LONG_DD',
 'USNG_FL_1K',
 'FDOE_MSID',
 'NCES_PUB',
 'NCES_PRIV',
 'FDOE_PRV',
 'SOURCE',
 'DESCRIPT',
 'FLAG',
 'UPDATE_DAY',
 'FGDLAQDATE',
 'AUTOID')

In [42]:
columns = ["NAME", 'OP_CLASS', 'ENROLLMENT', 'TYPE', 'TEACHERS']
school_arr = arcpy.da.FeatureClassToNumPyArray(school_fc, columns)

pd.DataFrame(school_arr, columns=columns)

,NAME,OP_CLASS,ENROLLMENT,TYPE,TEACHERS
0,GRACE CHRISTIAN SCHOOL OF ALACHUA CO.,PRIVATE,0.0,SENIOR HIGH,0.0
1,FAMILY LIFE ACADEMY,PRIVATE,0.0,COMBINATION ELEMENTARY & SECONDARY,0.0
2,FOREST GROVE CHRISTIAN ACADEMY,PRIVATE,53.0,COMBINATION ELEMENTARY & SECONDARY,9.4
3,VAISHNAVA ACADEMY FOR GIRLS,PRIVATE,19.0,COMBINATION JR. HIGH & SENIOR HIGH,0.0
4,BHAKTIVEDANTA ACADEMY,PRIVATE,0.0,COMBINATION ELEMENTARY & MIDDLE,0.0
...,...,...,...,...,...
107,SANTA FE COLLEGE - CHARLES L BLOUNT DOWNTOWN C...,PUBLIC,0.0,COLLEGE/UNIVERSITY,0.0
108,SANTA FE COLLEGE - KIRKPATRICK CENTER,PUBLIC,0.0,COLLEGE/UNIVERSITY,0.0
109,ALACHUA COUNTY JAIL,PUBLIC,0.0,COMBINATION JR. HIGH & SENIOR HIGH,0.0
110,ALACHUA COUNTY STUDENT SERVICES/ MIGRANT/ VIR...,PUBLIC,0.0,MIGRANT EDUCATION PROGRAM,4.0


### 2.3 Preview a DataFrame

The `head` and `tail` functions will allow us to preview the top
and bottom of a DataFrame, respectively.

In [43]:
school_df = pd.DataFrame(school_arr, columns=columns)
school_df.head()

,NAME,OP_CLASS,ENROLLMENT,TYPE,TEACHERS
0,GRACE CHRISTIAN SCHOOL OF ALACHUA CO.,PRIVATE,0.0,SENIOR HIGH,0.0
1,FAMILY LIFE ACADEMY,PRIVATE,0.0,COMBINATION ELEMENTARY & SECONDARY,0.0
2,FOREST GROVE CHRISTIAN ACADEMY,PRIVATE,53.0,COMBINATION ELEMENTARY & SECONDARY,9.4
3,VAISHNAVA ACADEMY FOR GIRLS,PRIVATE,19.0,COMBINATION JR. HIGH & SENIOR HIGH,0.0
4,BHAKTIVEDANTA ACADEMY,PRIVATE,0.0,COMBINATION ELEMENTARY & MIDDLE,0.0


And, by default (without giving an argument), both functions return 5 rows.

In [44]:
school_df.tail()

,NAME,OP_CLASS,ENROLLMENT,TYPE,TEACHERS
107,SANTA FE COLLEGE - CHARLES L BLOUNT DOWNTOWN C...,PUBLIC,0.0,COLLEGE/UNIVERSITY,0.0
108,SANTA FE COLLEGE - KIRKPATRICK CENTER,PUBLIC,0.0,COLLEGE/UNIVERSITY,0.0
109,ALACHUA COUNTY JAIL,PUBLIC,0.0,COMBINATION JR. HIGH & SENIOR HIGH,0.0
110,ALACHUA COUNTY STUDENT SERVICES/ MIGRANT/ VIR...,PUBLIC,0.0,MIGRANT EDUCATION PROGRAM,4.0
111,OAK HALL LOWER SCHOOL,PRIVATE,365.0,ELEMENTARY,16.0


But, we can specify the number we want to preview.

In [45]:
school_df.head(10)

,NAME,OP_CLASS,ENROLLMENT,TYPE,TEACHERS
0,GRACE CHRISTIAN SCHOOL OF ALACHUA CO.,PRIVATE,0.0,SENIOR HIGH,0.0
1,FAMILY LIFE ACADEMY,PRIVATE,0.0,COMBINATION ELEMENTARY & SECONDARY,0.0
2,FOREST GROVE CHRISTIAN ACADEMY,PRIVATE,53.0,COMBINATION ELEMENTARY & SECONDARY,9.4
3,VAISHNAVA ACADEMY FOR GIRLS,PRIVATE,19.0,COMBINATION JR. HIGH & SENIOR HIGH,0.0
4,BHAKTIVEDANTA ACADEMY,PRIVATE,0.0,COMBINATION ELEMENTARY & MIDDLE,0.0
5,DESTINY CHRISTIAN ACADEMY,PRIVATE,0.0,COMBINATION ELEMENTARY & MIDDLE,0.0
6,INCAF MONTESSORI SCHOOL,PRIVATE,0.0,ELEMENTARY,0.0
7,"GREAT AMERICAN VISIONS ENTERPRISES,INC",PRIVATE,0.0,COMBINATION ELEMENTARY & MIDDLE,0.0
8,JORDAN GLEN SCHOOL INC.,PRIVATE,115.0,COMBINATION ELEMENTARY & MIDDLE,14.5
9,QUEEN OF PEACE CATHOLIC ACADEMY,PRIVATE,358.0,COMBINATION ELEMENTARY & MIDDLE,28.4


### 2.4 Indexing and Slicing for DataFrame

Select single column returns a `pd.Series`.

In [46]:
school_df["NAME"]

0                  GRACE CHRISTIAN SCHOOL OF ALACHUA CO.
1                                    FAMILY LIFE ACADEMY
2                         FOREST GROVE CHRISTIAN ACADEMY
3                            VAISHNAVA ACADEMY FOR GIRLS
4                                  BHAKTIVEDANTA ACADEMY
                             ...                        
107    SANTA FE COLLEGE - CHARLES L BLOUNT DOWNTOWN C...
108                SANTA FE COLLEGE - KIRKPATRICK CENTER
109                                  ALACHUA COUNTY JAIL
110    ALACHUA  COUNTY STUDENT SERVICES/ MIGRANT/ VIR...
111                                OAK HALL LOWER SCHOOL
Name: NAME, Length: 112, dtype: object

Select multiple columns return a pandas **DataFrame**

In [47]:
school_df[["NAME", "ENROLLMENT"]] # multiple columns supplied as a list

,NAME,ENROLLMENT
0,GRACE CHRISTIAN SCHOOL OF ALACHUA CO.,0.0
1,FAMILY LIFE ACADEMY,0.0
2,FOREST GROVE CHRISTIAN ACADEMY,53.0
3,VAISHNAVA ACADEMY FOR GIRLS,19.0
4,BHAKTIVEDANTA ACADEMY,0.0
...,...,...
107,SANTA FE COLLEGE - CHARLES L BLOUNT DOWNTOWN C...,0.0
108,SANTA FE COLLEGE - KIRKPATRICK CENTER,0.0
109,ALACHUA COUNTY JAIL,0.0
110,ALACHUA COUNTY STUDENT SERVICES/ MIGRANT/ VIR...,0.0


Select row using the `.loc[]` function (**label/index-based**).
Note that `.loc` is followed by square brackets not parentheses.

In [48]:
school_df.loc[2]

NAME              FOREST GROVE CHRISTIAN ACADEMY
OP_CLASS                                 PRIVATE
ENROLLMENT                                  53.0
TYPE          COMBINATION ELEMENTARY & SECONDARY
TEACHERS                                     9.4
Name: 2, dtype: object

Select row using the `.iloc[]` function (**position/order-based**)

In [49]:
school_df.iloc[2]

NAME              FOREST GROVE CHRISTIAN ACADEMY
OP_CLASS                                 PRIVATE
ENROLLMENT                                  53.0
TYPE          COMBINATION ELEMENTARY & SECONDARY
TEACHERS                                     9.4
Name: 2, dtype: object

Use `set_index()` to choose a column as index.

- `inplace=True`
- reassign the result of `set_index()`

In [50]:
school_df.set_index('NAME').head()

,OP_CLASS,ENROLLMENT,TYPE,TEACHERS
NAME,,,,
GRACE CHRISTIAN SCHOOL OF ALACHUA CO.,PRIVATE,0.0,SENIOR HIGH,0.0
FAMILY LIFE ACADEMY,PRIVATE,0.0,COMBINATION ELEMENTARY & SECONDARY,0.0
FOREST GROVE CHRISTIAN ACADEMY,PRIVATE,53.0,COMBINATION ELEMENTARY & SECONDARY,9.4
VAISHNAVA ACADEMY FOR GIRLS,PRIVATE,19.0,COMBINATION JR. HIGH & SENIOR HIGH,0.0
BHAKTIVEDANTA ACADEMY,PRIVATE,0.0,COMBINATION ELEMENTARY & MIDDLE,0.0


In [51]:
school_df.head()

,NAME,OP_CLASS,ENROLLMENT,TYPE,TEACHERS
0,GRACE CHRISTIAN SCHOOL OF ALACHUA CO.,PRIVATE,0.0,SENIOR HIGH,0.0
1,FAMILY LIFE ACADEMY,PRIVATE,0.0,COMBINATION ELEMENTARY & SECONDARY,0.0
2,FOREST GROVE CHRISTIAN ACADEMY,PRIVATE,53.0,COMBINATION ELEMENTARY & SECONDARY,9.4
3,VAISHNAVA ACADEMY FOR GIRLS,PRIVATE,19.0,COMBINATION JR. HIGH & SENIOR HIGH,0.0
4,BHAKTIVEDANTA ACADEMY,PRIVATE,0.0,COMBINATION ELEMENTARY & MIDDLE,0.0


In [52]:
school_df.set_index('NAME', inplace=True) # or
# school_df = school_df.set_index('NAME')

In [37]:
school_df.head()

,OP_CLASS,ENROLLMENT,TYPE,TEACHERS
NAME,,,,
GRACE CHRISTIAN SCHOOL OF ALACHUA CO.,PRIVATE,0.0,SENIOR HIGH,0.0
FAMILY LIFE ACADEMY,PRIVATE,0.0,COMBINATION ELEMENTARY & SECONDARY,0.0
FOREST GROVE CHRISTIAN ACADEMY,PRIVATE,53.0,COMBINATION ELEMENTARY & SECONDARY,9.4
VAISHNAVA ACADEMY FOR GIRLS,PRIVATE,19.0,COMBINATION JR. HIGH & SENIOR HIGH,0.0
BHAKTIVEDANTA ACADEMY,PRIVATE,0.0,COMBINATION ELEMENTARY & MIDDLE,0.0


In [53]:
school_df.loc['FOREST GROVE CHRISTIAN ACADEMY']

OP_CLASS                                 PRIVATE
ENROLLMENT                                  53.0
TYPE          COMBINATION ELEMENTARY & SECONDARY
TEACHERS                                     9.4
Name: FOREST GROVE CHRISTIAN ACADEMY, dtype: object

In [54]:
school_df.loc[:, 'ENROLLMENT']

NAME
GRACE CHRISTIAN SCHOOL OF ALACHUA CO.                                                                 0.0
FAMILY LIFE ACADEMY                                                                                   0.0
FOREST GROVE CHRISTIAN ACADEMY                                                                       53.0
VAISHNAVA ACADEMY FOR GIRLS                                                                          19.0
BHAKTIVEDANTA ACADEMY                                                                                 0.0
                                                                                                    ...  
SANTA FE COLLEGE - CHARLES L BLOUNT DOWNTOWN CENTER                                                   0.0
SANTA FE COLLEGE - KIRKPATRICK CENTER                                                                 0.0
ALACHUA COUNTY JAIL                                                                                   0.0
ALACHUA  COUNTY STUDENT SERVICES/ MIGRANT

`.iloc` for both row indices and column indices.

In [55]:
school_df.iloc[10:20, 1:4]

,ENROLLMENT,TYPE,TEACHERS
NAME,,,
THE ROCK SCHOOL,207.0,COMBINATION ELEMENTARY & SECONDARY,17.9
CHRISTIAN LIFE ACADEMY,52.0,COMBINATION ELEMENTARY & SECONDARY,5.9
SAINT FRANCIS CATHOLIC HIGH SCHOOL,260.0,SENIOR HIGH,19.8
COUNTRYSIDE CHRISTIAN SCHOOL,106.0,COMBINATION ELEMENTARY & SECONDARY,6.5
TRILOGY SCHOOL OF LEARNING ALTERNATIVE,84.0,COMBINATION ELEMENTARY & SECONDARY,11.5
MILLHOPPER MONTESSORI SCHOOL,209.0,COMBINATION ELEMENTARY & MIDDLE,19.1
BNAI ISRAEL DAY SCHOOL,22.0,ELEMENTARY,0.0
GAINESVILLE CONDUCTIVE EDUCATION ACADEMY,0.0,COMBINATION ELEMENTARY & SECONDARY,0.0
BRENTWOOD SCHOOL,246.0,ELEMENTARY,10.8


Similar to indexing on a 2-d NumPy array, we can access individual values
of `DataFrame` based on row index and column index.
Below is an example of finding out _The Rock School_'s (row) size of Enrollment (column).

In [56]:
school_df.loc['THE ROCK SCHOOL', 'ENROLLMENT']

207.0

## 3. Selection Query on DataFrame

A single condition returns a Series of booleans again with the same index

In [57]:
school_df["ENROLLMENT"] > 0

NAME
GRACE CHRISTIAN SCHOOL OF ALACHUA CO.                                                               False
FAMILY LIFE ACADEMY                                                                                 False
FOREST GROVE CHRISTIAN ACADEMY                                                                       True
VAISHNAVA ACADEMY FOR GIRLS                                                                          True
BHAKTIVEDANTA ACADEMY                                                                               False
                                                                                                    ...  
SANTA FE COLLEGE - CHARLES L BLOUNT DOWNTOWN CENTER                                                 False
SANTA FE COLLEGE - KIRKPATRICK CENTER                                                               False
ALACHUA COUNTY JAIL                                                                                 False
ALACHUA  COUNTY STUDENT SERVICES/ MIGRANT

In [58]:
school_df.ENROLLMENT.head()

NAME
GRACE CHRISTIAN SCHOOL OF ALACHUA CO.     0.0
FAMILY LIFE ACADEMY                       0.0
FOREST GROVE CHRISTIAN ACADEMY           53.0
VAISHNAVA ACADEMY FOR GIRLS              19.0
BHAKTIVEDANTA ACADEMY                     0.0
Name: ENROLLMENT, dtype: float64

In [59]:
school_df.ENROLLMENT.index.name == school_df.index.name

True

In [60]:
school_df.index.values

array(['GRACE CHRISTIAN SCHOOL OF ALACHUA CO.', 'FAMILY LIFE ACADEMY',
       'FOREST GROVE CHRISTIAN ACADEMY', 'VAISHNAVA ACADEMY FOR GIRLS',
       'BHAKTIVEDANTA ACADEMY', 'DESTINY CHRISTIAN ACADEMY',
       'INCAF MONTESSORI SCHOOL',
       'GREAT AMERICAN VISIONS ENTERPRISES,INC',
       'JORDAN GLEN SCHOOL INC.', 'QUEEN OF PEACE CATHOLIC ACADEMY',
       'THE ROCK SCHOOL', 'CHRISTIAN LIFE ACADEMY',
       'SAINT FRANCIS CATHOLIC HIGH SCHOOL',
       'COUNTRYSIDE CHRISTIAN SCHOOL',
       'TRILOGY SCHOOL OF LEARNING ALTERNATIVE',
       'MILLHOPPER MONTESSORI SCHOOL', 'BNAI ISRAEL DAY SCHOOL',
       'GAINESVILLE CONDUCTIVE EDUCATION ACADEMY', 'BRENTWOOD SCHOOL',
       'CORNERSTONE ACADEMY', 'WESTWOOD HILLS CHRISTIAN SCHOOL',
       'FLOWERS MONTESSORI SCHOOL', 'Z.L. SUNG S.D.A. SCHOOL',
       'GAINESVILLE CONDUCTIVE EDUCATION ACADEMY',
       'KIDS N ALL CHRISTIAN ACADEMY', 'OAK HALL SCHOOL',
       'FREEDOM CHRISTIAN ACADEMY', 'GAINESVILLE COUNTRY DAY SCHOOL',
       'FAITH TA

Similar to NumPy, we can use **boolean indexing** for Pandas as well,
in which we create a condition (return Booleans in a Series) and then use it
to filter the original DataFrame.

In [62]:
school_df[school_df["ENROLLMENT"] > 0] # without loc

,OP_CLASS,ENROLLMENT,TYPE,TEACHERS
NAME,,,,
FOREST GROVE CHRISTIAN ACADEMY,PRIVATE,53.0,COMBINATION ELEMENTARY & SECONDARY,9.4
VAISHNAVA ACADEMY FOR GIRLS,PRIVATE,19.0,COMBINATION JR. HIGH & SENIOR HIGH,0.0
JORDAN GLEN SCHOOL INC.,PRIVATE,115.0,COMBINATION ELEMENTARY & MIDDLE,14.5
QUEEN OF PEACE CATHOLIC ACADEMY,PRIVATE,358.0,COMBINATION ELEMENTARY & MIDDLE,28.4
THE ROCK SCHOOL,PRIVATE,207.0,COMBINATION ELEMENTARY & SECONDARY,17.9
...,...,...,...,...
"FLORIDA SIATECH AT GAINESVILLE,INC.",PUBLIC,202.0,SENIOR HIGH,5.0
SHELL ELEMENTARY SCHOOL,PUBLIC,193.0,ELEMENTARY,16.0
HAWTHORNE JR/SR HIGH SCHOOL,PUBLIC,395.0,COMBINATION JR. HIGH & SENIOR HIGH,23.0


In [61]:
school_df.loc[school_df["ENROLLMENT"] > 0] # with loc (recommended)

,OP_CLASS,ENROLLMENT,TYPE,TEACHERS
NAME,,,,
FOREST GROVE CHRISTIAN ACADEMY,PRIVATE,53.0,COMBINATION ELEMENTARY & SECONDARY,9.4
VAISHNAVA ACADEMY FOR GIRLS,PRIVATE,19.0,COMBINATION JR. HIGH & SENIOR HIGH,0.0
JORDAN GLEN SCHOOL INC.,PRIVATE,115.0,COMBINATION ELEMENTARY & MIDDLE,14.5
QUEEN OF PEACE CATHOLIC ACADEMY,PRIVATE,358.0,COMBINATION ELEMENTARY & MIDDLE,28.4
THE ROCK SCHOOL,PRIVATE,207.0,COMBINATION ELEMENTARY & SECONDARY,17.9
...,...,...,...,...
"FLORIDA SIATECH AT GAINESVILLE,INC.",PUBLIC,202.0,SENIOR HIGH,5.0
SHELL ELEMENTARY SCHOOL,PUBLIC,193.0,ELEMENTARY,16.0
HAWTHORNE JR/SR HIGH SCHOOL,PUBLIC,395.0,COMBINATION JR. HIGH & SENIOR HIGH,23.0


For multiple conditions, use **parentheses** to enclose each condition and then connect
multiple conditions by either one of the following symbols.

- **AND** operation: `&`
- **OR** operation: `|`

Let's find out schools with both **positive enrollment** and **positive number of teachers**.

In [63]:
school_df[(school_df["ENROLLMENT"] > 0) & (school_df["TEACHERS"] > 0)]

,OP_CLASS,ENROLLMENT,TYPE,TEACHERS
NAME,,,,
FOREST GROVE CHRISTIAN ACADEMY,PRIVATE,53.0,COMBINATION ELEMENTARY & SECONDARY,9.4
JORDAN GLEN SCHOOL INC.,PRIVATE,115.0,COMBINATION ELEMENTARY & MIDDLE,14.5
QUEEN OF PEACE CATHOLIC ACADEMY,PRIVATE,358.0,COMBINATION ELEMENTARY & MIDDLE,28.4
THE ROCK SCHOOL,PRIVATE,207.0,COMBINATION ELEMENTARY & SECONDARY,17.9
CHRISTIAN LIFE ACADEMY,PRIVATE,52.0,COMBINATION ELEMENTARY & SECONDARY,5.9
...,...,...,...,...
"FLORIDA SIATECH AT GAINESVILLE,INC.",PUBLIC,202.0,SENIOR HIGH,5.0
SHELL ELEMENTARY SCHOOL,PUBLIC,193.0,ELEMENTARY,16.0
HAWTHORNE JR/SR HIGH SCHOOL,PUBLIC,395.0,COMBINATION JR. HIGH & SENIOR HIGH,23.0


In [64]:
school_df[(school_df["ENROLLMENT"] > 0) | (school_df["TEACHERS"] > 0)]

,OP_CLASS,ENROLLMENT,TYPE,TEACHERS
NAME,,,,
FOREST GROVE CHRISTIAN ACADEMY,PRIVATE,53.0,COMBINATION ELEMENTARY & SECONDARY,9.4
VAISHNAVA ACADEMY FOR GIRLS,PRIVATE,19.0,COMBINATION JR. HIGH & SENIOR HIGH,0.0
JORDAN GLEN SCHOOL INC.,PRIVATE,115.0,COMBINATION ELEMENTARY & MIDDLE,14.5
QUEEN OF PEACE CATHOLIC ACADEMY,PRIVATE,358.0,COMBINATION ELEMENTARY & MIDDLE,28.4
THE ROCK SCHOOL,PRIVATE,207.0,COMBINATION ELEMENTARY & SECONDARY,17.9
...,...,...,...,...
SHELL ELEMENTARY SCHOOL,PUBLIC,193.0,ELEMENTARY,16.0
HAWTHORNE JR/SR HIGH SCHOOL,PUBLIC,395.0,COMBINATION JR. HIGH & SENIOR HIGH,23.0
WALDO COMMUNITY SCHOOL,PUBLIC,215.0,ELEMENTARY,17.0


## 4. `groupby()` function

[groupby](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) is a
useful function to aggregate data.

A `groupby` operation involves some combination of

- splitting the object,
- **applying a function**, and
- combining the results.

This can be used to group large amounts of data and **compute operations on these groups**
to understand **subgroups** in the data.

In [75]:
school_df = school_df.loc[(school_df["ENROLLMENT"] > 0) & (school_df["TEACHERS"])]

Without the aggregation function, the return is simply a `groupby` object.

In [76]:
school_df.groupby("TYPE")

In [77]:
by_type = school_df.groupby("TYPE")
by_type.sum()

,ENROLLMENT,TEACHERS
TYPE,,
COMBINATION ELEMENTARY & MIDDLE,2664.0,176.0
COMBINATION ELEMENTARY & SECONDARY,3094.0,323.3
COMBINATION JR. HIGH & SENIOR HIGH,768.0,63.0
ELEMENTARY,13231.0,961.3
MIDDLE/JR. HIGH,4844.0,286.0
PRE-KINDERGARTEN,163.0,20.0
PRE-KINDERGARTEN-KINDERGARTEN,45.0,6.1
SENIOR HIGH,7885.0,390.8


In [78]:
by_type.max()

,OP_CLASS,ENROLLMENT,TEACHERS
TYPE,,,
COMBINATION ELEMENTARY & MIDDLE,PUBLIC,938.0,54.0
COMBINATION ELEMENTARY & SECONDARY,PUBLIC,1139.0,122.0
COMBINATION JR. HIGH & SENIOR HIGH,PUBLIC,395.0,23.0
ELEMENTARY,PUBLIC,836.0,56.0
MIDDLE/JR. HIGH,PUBLIC,1047.0,62.0
PRE-KINDERGARTEN,PUBLIC,100.0,15.0
PRE-KINDERGARTEN-KINDERGARTEN,PRIVATE,45.0,6.1
SENIOR HIGH,PUBLIC,2221.0,100.0


In [79]:
school_df.groupby("TYPE").mean()

,ENROLLMENT,TEACHERS
TYPE,,
COMBINATION ELEMENTARY & MIDDLE,333.000000,22.000000
COMBINATION ELEMENTARY & SECONDARY,238.000000,24.869231
COMBINATION JR. HIGH & SENIOR HIGH,192.000000,15.750000
ELEMENTARY,400.939394,29.130303
MIDDLE/JR. HIGH,605.500000,35.750000
PRE-KINDERGARTEN,81.500000,10.000000
PRE-KINDERGARTEN-KINDERGARTEN,45.000000,6.100000
SENIOR HIGH,1126.428571,55.828571


`agg` stands for "aggregation" allows us to specify different aggregation functions
for different columns.

In [70]:
by_type.agg({'ENROLLMENT': np.sum, 'TEACHERS': np.mean, 'OP_CLASS': 'count'})

,ENROLLMENT,TEACHERS,OP_CLASS
TYPE,,,
COMBINATION ELEMENTARY & MIDDLE,2664.0,22.000000,8
COMBINATION ELEMENTARY & SECONDARY,3094.0,24.869231,13
COMBINATION JR. HIGH & SENIOR HIGH,768.0,15.750000,4
ELEMENTARY,13231.0,29.130303,33
MIDDLE/JR. HIGH,4844.0,35.750000,8
PRE-KINDERGARTEN,163.0,10.000000,2
PRE-KINDERGARTEN-KINDERGARTEN,45.0,6.100000,1
SENIOR HIGH,7885.0,55.828571,7
